# Automatically create Images of Code Snippets using Selenium + Carbon
## Setup
First we define the CARBON Url and specify the class of the Export Button from the carbon.now.sh website. We then define a function that generates the images by making the code url safe (escaping characters), using selenium to open the carbon website and click on the export button, then rename the downloaded file.

In [17]:
from time import sleep
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By

CARBON = 'https://carbon.now.sh/?' \
         'bg=rgba%28201%2C201%2C201%2C1%29' \
         '&t=one-light' \
         '&wt=none' \
         '&l=python' \
         '&width=879' \
         '&ds=false' \
         '&dsyoff=20px' \
         '&dsblur=68px' \
         '&wc=false' \
         '&wa=true' \
         '&pv=0px' \
         '&ph=0px' \
         '&ln=false' \
         '&fl=1' \
         '&fm=Source+Code+Pro' \
         '&fs=16px' \
         '&lh=199%25' \
         '&si=false' \
         '&es=2x' \
         '&wm=false' \
         '&code={code}'

# class name of Export Button
EXPORT_BUTTON_CLASS = "jsx-2184717013"

def generate_code_snippet_image(code: str, file: str):
    # make code url safe
    safe_code = urllib.parse.quote_plus(code)

    # set carbon url containing code
    url = CARBON.format(code=safe_code)

    # selenium webdriver
    driver = webdriver.Chrome() # setup chrome
    driver.get(url) # load carbon webpage
    driver.find_element(By.CLASS_NAME, EXPORT_BUTTON_CLASS).click() # click on export image button

    # wait while downloading and rename file
    sleep(3)
    downloads_folder = f"{Path.home()}/Downloads"
    os.rename(os.path.join(downloads_folder, 'carbon.png'), os.path.join(downloads_folder,f"{file}.png"))
    print(f"generated {file}.png")

## Iterating over multiple files
We can iterate over all files of the specified `parent_dir` and its subfolders to call the `generate_code_snippet_image()` function. I'm furthermore splitting the file contents into two images, seperated by the delimiter "# # # # # # # # # # # #".

In [ ]:
import os
from pathlib import Path
parent_dir = "../github_code_snippets_s5_final/"

for subdir, dirs, files in os.walk(parent_dir):
    for file in files:
        # ignore hidden & readme files
        if file.startswith(".") or file == "README.md":
            continue

        # read code snippet and generate image
        f = open(os.path.join(subdir, file), "r")
        code = f.read()

        try:
            # delimiter_index = code.find("# # # # # # # # # # # #")
            delimiter = "# # # # # delimiter # # # # # # #"
            delimiter_index = code.find(delimiter)
            delimiter_offset = len(delimiter)+1
            generate_code_snippet_image(code[:delimiter_index], f"{file[:-3]}-code-final")
            generate_code_snippet_image(code[delimiter_index+delimiter_offset:], f"{file[:-3]}-task-final")
        except Exception as e:
            print(f"Error occurred for {file}: {e}")